<a href="https://colab.research.google.com/github/Godofin/data_science/blob/main/WorldCup1930~2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 00 - Importando Dataset e Bibliotecas

In [1]:
import pandas as pd

In [10]:
results = pd.read_csv('/content/results.csv', parse_dates=['date'])

In [11]:
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


# 01 - Tratando Informações

In [12]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        44206 non-null  datetime64[ns]
 1   home_team   44206 non-null  object        
 2   away_team   44206 non-null  object        
 3   home_score  44202 non-null  float64       
 4   away_score  44202 non-null  float64       
 5   tournament  44206 non-null  object        
 6   city        44206 non-null  object        
 7   country     44206 non-null  object        
 8   neutral     44206 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(5)
memory usage: 2.7+ MB


## Maneiras de filtar dados utilizando data

Filtrando por Ano

In [13]:
results['date'].dt.year

0        1872
1        1873
2        1874
3        1875
4        1876
         ... 
44201    2022
44202    2022
44203    2022
44204    2022
44205    2022
Name: date, Length: 44206, dtype: int64

Filtrando por Mês

In [14]:
results['date'].dt.month

0        11
1         3
2         3
3         3
4         3
         ..
44201    12
44202    12
44203    12
44204    12
44205    12
Name: date, Length: 44206, dtype: int64

Filtrando por Dia

In [15]:
results['date'].dt.day

0        30
1         8
2         7
3         6
4         4
         ..
44201     1
44202     2
44203     2
44204     2
44205     2
Name: date, Length: 44206, dtype: int64

## Extraindo dados da copa

Filtrando apenas por jogos da copa do mundo

In [18]:
results_world_cup = results[results['tournament']=='FIFA World Cup']

In [19]:
results_world_cup.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
1311,1930-07-13,Belgium,United States,0.0,3.0,FIFA World Cup,Montevideo,Uruguay,True
1312,1930-07-13,France,Mexico,4.0,1.0,FIFA World Cup,Montevideo,Uruguay,True
1313,1930-07-14,Brazil,Yugoslavia,1.0,2.0,FIFA World Cup,Montevideo,Uruguay,True
1314,1930-07-14,Peru,Romania,1.0,3.0,FIFA World Cup,Montevideo,Uruguay,True
1315,1930-07-15,Argentina,France,1.0,0.0,FIFA World Cup,Montevideo,Uruguay,True


Criando uma coluna com o ano do jogo

In [21]:
results_world_cup['year'] = results_world_cup['date'].dt.year

<ipython-input-21-d853c8255f49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_world_cup['year'] = results_world_cup['date'].dt.year


Criando um data frame com resultados fora e dentro de casa

In [35]:
home_scores = results_world_cup[['home_team', 'home_score', 'year']].rename({'home_team':'team','home_score':'score'}, axis='columns')
away_scores = results_world_cup[['away_team', 'away_score', 'year']].rename({'away_team':'team','away_score':'score'}, axis='columns')

Concatenando os data frames

In [42]:
scores_world_cup = pd.concat([home_scores, away_scores]).reset_index(drop=True).fillna(0)
# reset_index(drop=true) -> reseta o index dropa o antigo
# fillna(0) -> transforma os NaN em 0

In [43]:
scores_world_cup

,team,score,year
0,Belgium,0.0,1930
1,France,4.0,1930
2,Brazil,1.0,1930
3,Peru,1.0,1930
4,Argentina,1.0,1930
...,...,...,...
1891,Morocco,2.0,2022
1892,Switzerland,0.0,2022
1893,Brazil,0.0,2022
1894,Uruguay,0.0,2022


In [67]:
score_world_cup_year = scores_world_cup.groupby(['team', 'year']).score.sum().unstack().fillna(0).cumsum(axis=1).reset_index()

Importando dados da confederação e bandeiras

In [61]:
flags = pd.read_csv('/content/ountries-fifa-flags.csv')

In [68]:
score_world_cup_year .merge(flags, left_on='team', right_on='country', how='left')

,team,1930,1934,1938,1950,1954,1958,1962,1966,1970,...,2002,2006,2010,2014,2018,2022,code,country,confederation,url
0,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,6.0,13.0,13.0,13.0,ALG,Algeria,CAF,https://countryflagsapi.com/png/algeria
1,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,ANT,Angola,CAF,https://countryflagsapi.com/png/angola
2,Argentina,18.0,20.0,20.0,20.0,20.0,25.0,27.0,31.0,31.0,...,102.0,113.0,123.0,131.0,137.0,142.0,ARG,Argentina,CONMEBOL,https://countryflagsapi.com/png/argentina
3,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,8.0,11.0,13.0,16.0,AUS,Australia,OFC,https://countryflagsapi.com/png/australia
4,Austria,0.0,7.0,7.0,7.0,24.0,26.0,26.0,26.0,26.0,...,43.0,43.0,43.0,43.0,43.0,43.0,AUT,Austria,UEFA,https://countryflagsapi.com/png/austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,United Arab Emirates,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,UAE,United Arab Emirates,AFC,https://countryflagsapi.com/png/united%20arab%...
78,United States,7.0,8.0,8.0,12.0,12.0,12.0,12.0,12.0,12.0,...,25.0,27.0,32.0,37.0,37.0,39.0,NaN,NaN,NaN,NaN
79,Uruguay,15.0,15.0,15.0,30.0,46.0,46.0,50.0,52.0,56.0,...,65.0,65.0,76.0,80.0,87.0,87.0,URU,Uruguay,CONMEBOL,https://countryflagsapi.com/png/uruguay
80,Wales,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,5.0,WAL,Wales,UEFA,https://countryflagsapi.com/png/wales


# 02 - Extraindo Dados tratados

In [69]:
score_world_cup_year .merge(flags, left_on='team', right_on='country', how='left').to_csv('world-cup.csv', index=False)